# Advanced simulation

In essence, a Model object is able to change the state of the system given a sample and evaluate certain metrics.

![Simple Model](Model_Simple_UML.png "Model Simple UML")

Model objects are able to drastically cut simulation time by sorting the samples to minimize perturbations to the system between simulations. This decreases the number of iterations required to solve recycle systems. The following examples show how Model objects can be used.


### Create a model object

**Model objects are used to evaluate metrics around multiple parameters of a system.**

Create a Model object of the lipidcane biorefinery with internal rate of return as a metric:

In [1]:
from biosteam.biorefineries import lipidcane as lc
import biosteam as bst
solve_IRR = lc.lipidcane_tea.solve_IRR
metrics = bst.Metric('IRR', solve_IRR),
model = bst.Model(lc.lipidcane_sys, metrics)


The Model object begins with no paramters: 

In [2]:
model

Model: Biorefinery IRR
 (No parameters)


Note: Here we defined only one metric, but more metrics are possible.

### Add design parameters

**A design parameter is a Unit attribute that changes design requirements but does not affect mass and energy balances.**

Add number of fermentation reactors as a "design" parameter:

In [3]:
R301 = bst.find.unit.R301 # The Fermentation Unit
@model.parameter(element=R301, kind='design', name='Number of reactors')
def set_N_reactors(N):
    R301.N = N

The decorator returns a Parameter object and adds it to the model:

In [4]:
set_N_reactors

<Parameter: [Fermentation-R301] Number of reactors>

Calling a Parameter object will update the parameter and results:

In [5]:
set_N_reactors(5)
print('Puchase cost at 5 reactors: ' + str(R301.purchase_cost))
set_N_reactors(8)
print('Puchase cost at 8 reactors: ' + str(R301.purchase_cost))

Puchase cost at 5 reactors: 2489477.4197166027
Puchase cost at 8 reactors: 2989917.5467727524


### Add cost parameters

**A cost parameter is a Unit attribute that affects cost but does not change design requirements.**

Add the fermentation unit base cost as a "cost" parameter:

In [6]:
@model.parameter(element=R301, kind='cost') # Note: name argument not given this time
def set_base_cost(cost):
    R301.cost_items['Reactors'].cost = cost

In [7]:
original = R301.cost_items['Reactors'].cost
set_base_cost(10e6)
print('Purchase cost at 10 million USD: ' + str(R301.purchase_cost))
set_base_cost(844e3)
print('Purchase cost at 844,000 USD: ' + str(R301.purchase_cost))

Purchase cost at 10 million USD: 23792335.704496454
Purchase cost at 844,000 USD: 2989917.5467727524


If the name was not defined, it defaults to the setter's signature:

In [8]:
set_base_cost

<Parameter: [Fermentation-R301] Cost>

### Add isolated parameters

**An isolated parameter should not affect Unit objects in any way.**

Add feedstock price as a "isolated" parameter:

In [9]:
lipid_cane = lc.lipid_cane # The feedstock stream
@model.parameter(element=lipid_cane, kind='isolated')
def set_feed_price(feedstock_price):
    lipid_cane.price = feedstock_price

### Add coupled parameters

**A coupled parameter affects mass and energy balances of the system.**

Add lipid fraction as a "coupled" parameter:

In [10]:
set_lipid_fraction = model.parameter(lc.set_lipid_fraction, element=lipid_cane, kind='coupled')

In [11]:
set_lipid_fraction(0.10)
print('IRR at 10% lipid: ' + str(solve_IRR()))
set_lipid_fraction(0.05)
print('IRR at 5% lipid: ' + str(solve_IRR()))

IRR at 10% lipid: 0.2005808257733235
IRR at 5% lipid: 0.15921216999065335


Add fermentation efficiency as a "coupled" parameter:

In [12]:
@model.parameter(element=R301, kind='coupled')
def set_fermentation_efficiency(efficiency):
    R301.efficiency = efficiency

### Evaluate metric given a sample

**The model can be called to evaluate a sample of parameters.**

All parameters are stored in the model with highly coupled parameters first:

In [13]:
model

Model: Biorefinery IRR
 Element:           Parameter:
 Stream-lipid cane  Lipid fraction
 Fermentation-R301  Efficiency
                    Number of reactors
                    Cost
 Stream-lipid cane  Feedstock price


Get all parameters as ordered in the model:

In [14]:
model.get_parameters()

(<Parameter: [Stream-lipid cane] Lipid fraction>,
 <Parameter: [Fermentation-R301] Efficiency>,
 <Parameter: [Fermentation-R301] Number of reactors>,
 <Parameter: [Fermentation-R301] Cost>,
 <Parameter: [Stream-lipid cane] Feedstock price>)

Evaluate sample:

In [15]:
model([0.05, 0.85, 8, 100000, 0.040])

[0.11942692089697049]

### Evaluate metric across samples

Evaluate at give parameter values:

In [16]:
import numpy as np
samples = np.array([(0.05, 0.85, 8, 100000, 0.040),
                    (0.05, 0.90, 7, 100000, 0.040),
                    (0.09, 0.95, 8, 100000, 0.042)])
model.load_samples(samples)
model.evaluate()
model.table # All evaluations are stored as a pandas DataFrame

Element  Stream-lipid cane Fermentation-R301                           \
Variable    Lipid fraction        Efficiency Number of reactors  Cost   
0                     0.05              0.85                  8 1e+05   
1                     0.05               0.9                  7 1e+05   
2                     0.09              0.95                  8 1e+05   

Element  Stream-lipid cane Biorefinery  
Variable   Feedstock price         IRR  
0                     0.04       0.119  
1                     0.04       0.131  
2                    0.042       0.167

Note that coupled parameters are on the left most columns, and are ordered from upstream to downstream (e.g. <Stream: Lipid cane> is upstream from <Fermentation: R301>)

### Evaluate multiple metrics

Reset the metrics to include total utility cost:

In [17]:
def total_utility_cost():
    """Return utility costs in 10^6 USD/yr"""
    return lc.lipidcane_tea.utility_cost / 10**6

# This time use detailed names and units for appearance
model.metrics = (bst.Metric('Internal rate of return', lc.lipidcane_tea.solve_IRR, '%'),
                 bst.Metric('Utility cost', total_utility_cost, 'USD/yr'))
model

Model: Biorefinery internal rate of return (%)
       Biorefinery utility cost (USD/yr)
 Element:           Parameter:
 Stream-lipid cane  Lipid fraction
 Fermentation-R301  Efficiency
                    Number of reactors
                    Cost
 Stream-lipid cane  Feedstock price


In [18]:
model.evaluate()
model.table

Element  Stream-lipid cane Fermentation-R301                           \
Variable    Lipid fraction        Efficiency Number of reactors  Cost   
0                     0.05              0.85                  8 1e+05   
1                     0.05               0.9                  7 1e+05   
2                     0.09              0.95                  8 1e+05   

Element  Stream-lipid cane Biorefinery                                       
Variable   Feedstock price         IRR Internal rate of return Utility cost  
0                     0.04       0.119                   0.119        -17.6  
1                     0.04       0.131                   0.131        -16.9  
2                    0.042       0.167                   0.167          -25

### Behind the scenes

![Model UML Diagram](Model_UML.png "Model UML")

Model objects work with the help of Block and Parameter objects that are able to tell the relative importance of parameters through the `element` it affects and the `kind` (how it affects the system). Before a new parameter is made, if its `kind` is "coupled", then the Model object creates a Block object that simulates only the objects affected by the parameter. The Block object, in turn, helps to create a Parameter object by passing its simulation method.